## Catalog builder

DEMO ...

In [ ]:
# Append base directory to the system path
import os
import sys

working_directory = os.getcwd()
base_directory = working_directory + '/..'

sys.path.append(base_directory)

In [ ]:
# Reload python modules automatically
%reload_ext autoreload
%autoreload 2

In [ ]:
### Explain connection details...

In [ ]:
# DB connection
# REVIEW:
# - conenction parameters are hardcoded and exposed
# - user permissions

import sqlalchemy
import sqlparse
import warnings
warnings.simplefilter("ignore", category=sqlalchemy.exc.SAWarning)

# module to facilitate the DB connection and the creation and deletion of the tables
from src import db
from src import sqlalchemy_extension as sql_op

from src import queries_y1 as qy1


url = sqlalchemy.engine.url.URL('postgresql', username='', password='',host='', port=)
dal = db.DbConnection()
dal.db_init(url)

### STEP 1 - Region selection

why(?) - physics explanation

Steps:
1. Select an ancillary map from the list above.
2. Build a sql statement. -show query before execution-
3. Create table from sql statement
4. Report info and statistics about the table -Product log-

In [ ]:
# Load tables and inspect columns
# REVIEW:
# - how to choose the available tables outside E2E env ? -> Use 
# - list availabe tables
table_mangle_i = sqlalchemy.Table("mangle_maps_4096_ring_frac_i_10029940", dal.meta, autoload=True, schema="y1a1_coadd_cosmos_d04").alias('table_mangle_i')

print("Inspecting columns from table:\n")
print(str(table_mangle_i.c.values()))

In [ ]:
# Regions with mangle detfract larger than 0.8 in i band.
from sqlalchemy.sql import and_, or_

# stm = sqlalchemy.select([table_mangle_i]).where(and_(table_mangle_i.c.signal >= 0.8, table_mangle_i.c.signal <= 0.9))
stm = sqlalchemy.select([table_mangle_i]).where(table_mangle_i.c.signal >= 0.8)
print(sqlparse.format(str(stm), reindent=True, keyword_case='upper'))

In [ ]:
%%time
dal.create_table('tmp_table_mangle_i', stm)

In [ ]:
data = dal.select_columns("tmp_table_mangle_i", ['signal'])

In [ ]:
# Statistics and relevant info HERE.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


plt.clf()
n, bins, patches = plt.hist(np.array(data), 25, facecolor='red', alpha=0.50)
plt.xlabel('X label')
plt.ylabel('Y label')
plt.show()

In [ ]:
# dal.delete_table('tmp_table_mangle_i')

In [ ]:
# Surface plots
from src import plots as pt

data = dal.select_columns('tmp_table_mangle_i', ['ra', 'dec', 'signal'])
pt.plot_map(data)

In [ ]:
table_bad_regions = sqlalchemy.Table("y1a1_coadd_cosmos_d04_bright_mask_4096_ring_10023572", dal.meta, autoload=True, schema="bright_mask").alias('table_bad_regions')
str(table_bad_regions.c.values())

In [ ]:
# filter condition
# 1 - Regions with bad astrometric colors
# 2 - Fainter 2MASS star region (8 < J < 12)
# 4 - Large nearby object (R3C catalog)
# 8 - Bright 2MASS star region (5 < J < 8)
# 16 - Near the LMC
# 32 - Yale Bright Star region
# 64 - High density of crazy colors
# 128 - Globular Clusters (William et al. 2010)
filters = [32, 8, 4, 2]

stm = sqlalchemy.select([table_bad_regions]).where(sql_op.BitwiseAnd(
                                    sqlalchemy.cast(table_bad_regions.c.signal, sqlalchemy.Integer),
                                    sum(filters)) > 0)
print(sqlparse.format(str(stm), reindent=True, keyword_case='upper'))

In [ ]:
%%time
dal.create_table('tmp_table_bad_regions', stm)

In [ ]:
data = dal.select_columns('tmp_table_bad_regions', ['ra', 'dec', 'signal'])
pt.plot_map(data)

In [ ]:
# Add comments.
table_depth_i = sqlalchemy.Table("y1a1_coadd_cosmos_d04_5_auto_4096_ring_i_10024579", dal.meta, autoload=True, schema="depth_maps").alias('table_mangle_i')
str(table_depth_i.c.values())

In [ ]:
# Inspect SQL
stm = sqlalchemy.select([table_depth_i]).where(table_depth_i.c.signal >= 22)
print(sqlparse.format(str(stm), reindent=True, keyword_case='upper'))

In [ ]:
%%time
dal.create_table('tmp_table_depth_i', stm)

In [ ]:
data = dal.select_columns('tmp_table_depth_i', ['signal'])

In [ ]:
plt.clf()
n, bins, patches = plt.hist(np.array(data), 50, facecolor='green', alpha=0.75)
plt.xlabel('data')
plt.ylabel('Count')
plt.show()

In [ ]:
data = dal.select_columns('tmp_table_depth_i', ['ra', 'dec', 'signal'])
pt.plot_map(data)

In [ ]:
# Footprint
input_params = {}
table_dependencies = {}

table_dependencies['good_regions'] = []
table_dependencies['bad_regions'] = []

table_dependencies['good_regions'].append({'schema': dal.schema_output, 'table': 'tmp_table_depth_i'})
table_dependencies['good_regions'].append({'schema': dal.schema_output, 'table': 'tmp_table_mangle_i'})
table_dependencies['bad_regions'].append({'schema': dal.schema_output, 'table': 'tmp_table_bad_regions'})

operation = qy1.Footprint(input_params, table_dependencies)
print(sqlparse.format(operation.sql(), reindent=True, keyword_case='upper'))

In [ ]:
%%time
operation.create_table('tmp_footprint')

In [ ]:
data = dal.select_columns('tmp_footprint', ['signal'])

In [ ]:
data = dal.select_columns('tmp_footprint', ['ra', 'dec', 'signal'])
pt.plot_map(data)

STEP 2 - Object SELECTION

Reduciton<br/>
Cuts<br/>
Bitmask<br/>
Column selection<br/>

In [ ]:
# Reduction
input_params = {}
table_dependencies = {}

input_params['coadd_objects_ring'] = {'schema': 'y1a1_coadd_cosmos_d04', 'table': 'coadd_objects_pixel_4096_ring_10029940'}
table_dependencies['footprint'] = {'schema': dal.schema_output, 'table': 'tmp_footprint'}

operation = qy1.Reduction(input_params, table_dependencies)
print(sqlparse.format(operation.sql(), reindent=True, keyword_case='upper'))

In [ ]:
%%time
operation.create_table('tmp_reduction')

In [ ]:
# Cuts
input_params = {}
table_dependencies = {}

input_params['coadd_objects'] = {'schema': 'y1a1_coadd_cosmos_d04', 'table': 'coadd_objects'}
input_params['mag_type'] = 'auto'
input_params['sextractor_flags'] = [0, 1, 2]
input_params['sextractor_bands'] = ['i'] 
input_params['additional_cuts'] = ['remove_bbj', 'spreaderr_model', 'bad_astronomic_colors']
input_params['niter_model'] = []
input_params['sn_cuts'] = {}
input_params['magnitude_limit'] = {'i': '22'}
input_params['bright_magnitude'] = {'i': '15'}
input_params['color_cuts'] = {'gr':[-2, 4], 'ri':[-2, 4], 'iz':[-2, 4], 'zy':[-2, 4]}

table_dependencies['reduction'] = {'schema': dal.schema_output, 'table': 'tmp_reduction'}

operation = qy1.Cuts(input_params, table_dependencies)
print(sqlparse.format(operation.sql(), reindent=True, keyword_case='upper'))

In [ ]:
%%time
operation.create_table('tmp_cuts')

In [ ]:
# Bitmask
input_params = {}
table_dependencies = {}

input_params['molygon_coadds'] = {'schema': 'y1a1_coadd_cosmos_d04', 'table': 'coadd_objects_molygon_10029940'}
input_params['molygon'] = {'schema': 'y1a1_coadd_cosmos_d04', 'table': 'molygon_10029940'}
input_params['mangle_bitmask'] = ['i']

table_dependencies['last_table'] = {'schema': dal.schema_output, 'table': 'tmp_cuts'}

operation = qy1.Bitmask(input_params, table_dependencies)
print(sqlparse.format(operation.sql(), reindent=True, keyword_case='upper'))

In [ ]:
%%time
operation.create_table('tmp_bitmask')

In [ ]:
# Object Selection
input_params = {}
table_dependencies = {}

input_params['coadd_objects'] = {'schema': 'y1a1_coadd_cosmos_d04', 'table': 'coadd_objects'}
input_params['columns_data_set'] = ["a_image", "alphawin_j2000_y", "bwin_image_r", "chi2_psf_r", "ra", "dec"]
input_params['columns_zero_point'] = []

table_dependencies['last_table'] = {'schema': dal.schema_output, 'table': 'tmp_bitmask'}
table_dependencies['zero_point'] = {'schema': '', 'table': ''}

operation = qy1.ObjectSelection(input_params, table_dependencies)
print(sqlparse.format(operation.sql(), reindent=True, keyword_case='upper'))

In [ ]:
%%time
operation.create_table('tmp_object_selection')

In [ ]:
data = dal.select_columns('tmp_object_selection', ['ra', 'dec'])
pt.plot_catalog(4096, data)